#**Computer Vision : Assignment 1**
**Group :** \
Duarte Ribeiro Afonso Branco (up201905327@up.pt) \
Manuel de Magalhães Carvalho Cerqueira da Silva (up201806391@up.pt)

#**Download Images:**

In [ ]:
!gdown --folder https://drive.google.com/drive/folders/1V7PZn9WeSbOEB-j4MHlwIk3eV3mOOvzZ?usp=sharing

**Note:** For some reason sometimes Colab decides it doesnt want to download images from the drive, and an error occours indicating that it doesnt have the correct premissions. If this happends go to **"Tempo de execução" > "Dessassociar e elimanar tempo de execução"**. If even this doesnt work please contact us.

# **Imports**

In [ ]:
import numpy as np
import cv2
import glob
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import math

from matplotlib.colors import ListedColormap
from collections import defaultdict
from scipy.stats import linregress
import json


# **Global auxiliary functions**


In [ ]:
def plotTwoImages(image1, image2, title1, title2):
  # Create a side-by-side plot with titles
  plt.close('all')
  fig, axes = plt.subplots(1, 2, figsize=(16, 8))

  axes[0].imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
  axes[0].set_title(title1)
  #axes[0].axis('off')
  axes[0].set_xlabel("X (pixels)")
  axes[0].set_ylabel("Y (pixels)")

  axes[1].imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
  axes[1].set_title(title2)
  #axes[1].axis('off')
  axes[1].set_xlabel("X (pixels)")
  axes[1].set_ylabel("Y (pixels)")

  plt.show()

def getImagesPathsList(folderName):
  imagesPathsList = glob.glob(folderName + '/*.png')

  if not imagesPathsList:
    raise Exception("Could not get the path for the images from folder:" + folderName)

  return imagesPathsList

def calcDiameter(area):
  return math.sqrt(area/(math.pi))*2

def circleArea(radius):
  area = math.pi * (radius ** 2)
  return area

def getNameFromPath(imagePath):
      return imagePath.split('/')[-1]


#**Exercise a)**
Calibrate the intrinsic parameters and lens distortion of the camera.

In [ ]:
def calculateOptimalUndistortion(image, height, width, intrinsicMatrix, distortionCoeffs):

  alpha = 1
  newIntrinsicMatrix, validPixRoi = cv2.getOptimalNewCameraMatrix(intrinsicMatrix, distortionCoeffs, (width, height), alpha, (width, height))

  undistortedImage = None
  undistortedImage = cv2.undistort(image, intrinsicMatrix, distortionCoeffs, None, newIntrinsicMatrix)

  x, y, w, h = validPixRoi
  undistortedImage = undistortedImage[y:y+h, x:x+w]

  return undistortedImage

def findChessboardCorners(calibrationImage, patternSize):
    calibrationImageGray = cv2.cvtColor(calibrationImage, cv2.COLOR_BGR2GRAY)
    retval, corners = cv2.findChessboardCorners(calibrationImageGray, patternSize)
    if retval != 0:
        corners = cv2.cornerSubPix(calibrationImageGray, corners, (11, 11), (-1, -1), (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001))
        return retval, corners
    else:
        return 0, None

In [ ]:
patternSize = (7, 4)
objectPoints = np.zeros((patternSize[0] * patternSize[1], 3), np.float32)
objectPoints[:, :2] = np.mgrid[0:patternSize[0], 0:patternSize[1]].T.reshape(-1, 2)

allCalibrationImageCorners = []
allObjectPoints = []
allImagePoints = []

calibrationImagePaths = getImagesPathsList("/content/Assignment1/IntrinsicCalibration")

for calibrationImagePath in calibrationImagePaths:
    calibrationImage = cv2.imread(calibrationImagePath)
    retval, corners = findChessboardCorners(calibrationImage, patternSize)

    if retval != 0:
        allObjectPoints.append(objectPoints)
        allImagePoints.append(corners)

        cv2.drawChessboardCorners(calibrationImage, patternSize, corners, retval)
        allCalibrationImageCorners.append(calibrationImage)
    else:
        print("Couldn't find all corners in:", calibrationImagePath)

height, width, channels = cv2.imread(calibrationImagePaths[0]).shape

retval, INTRINSIC_MATRIX, DISTORTION_COEFFS, rvecs, tvecs = cv2.calibrateCamera(allObjectPoints, allImagePoints, (width, height), None, None)

for calibrationImageCorners in allCalibrationImageCorners:
    undistortedCalibrationImageCorners = calculateOptimalUndistortion(calibrationImageCorners, height, width, INTRINSIC_MATRIX, DISTORTION_COEFFS)
    plotTwoImages(calibrationImageCorners, undistortedCalibrationImageCorners, "Distorted", "Undistorted")

print("Intrinsic Matrix:\n", INTRINSIC_MATRIX)
print("Lens Distortion Coefficients:\n", DISTORTION_COEFFS)

**Correct camera distortion auxiliary function**

In [ ]:
def correctCameraDistortion(image):
  height, width, _ = image.shape
  return calculateOptimalUndistortion(image, height, width, INTRINSIC_MATRIX, DISTORTION_COEFFS)

Matrices obtained from code:

**Intrinsic Matrix**:
\
\begin{bmatrix}
1.32164961 \times 10^3 & 0 & 9.88299668 \times 10^2 \\
0 & 1.32444624 \times 10^3 & 6.42259078 \times 10^2 \\
0 & 0 & 1.00000000 \times 10^0
\end{bmatrix}


**Lens Distortion Coefficients**:
\
\begin{bmatrix}
-3.44269790 \times 10^{-1} & 9.00566069 \times 10^{-2} & 9.45585538 \times 10^{-5} & -3.97940158 \times 10^{-4} & -3.15658827 \times 10^{-3}
\end{bmatrix}

\

# **Exercise b)**
Use the “External_calib_img” to calculate the conversion rate between pixel to millimeter. Discuss if the conversion rate obtained is uniform across the entire image.

**Awnser:**
\
The conversion ratio was obtain by measuring the pixel coordinates of the adjacent centimeters in the undistorted ruler image. The distances between these coordinates are plotted in a graph and then fitted with a linear regression. This way it's possible to characterize these distances with the X and Y coordinates. To obtain the pixel to millimeter ratio the Pythagorean Theorem is used to relate both distances.
\
This ratio is liniarized only arround the selected ruller points. Since the camera point of view is croocked, for example, in the top left of the image less pixels are necessary to represent one millimiter, as shown in the graphs.


In [ ]:
externalImg = cv2.imread('/content/Assignment1/ExternalCalibration/External_calib_img.png')

height, width, _ = externalImg.shape

# Define the minimum area for connected components
connectedComponentAreaMin = 10

undistExternalImg = correctCameraDistortion(externalImg)

plotTwoImages(externalImg, undistExternalImg, "Distorted", "Undistorted")

# Save the undistorted image
cv2.imwrite('undistExternalImg.png', undistExternalImg)

# Define the points' coordinates
coordinatesList = [(499, 584), (521, 569), (543, 556), (567, 541), (590, 527),
                   (614, 512), (640, 498), (661, 483), (686, 468), (709, 452), (735, 438),
                   (759, 423), (784, 408), (812, 393), (835, 377), (861, 361),
                   (886, 344), (912, 328), (938, 311), (964, 295), (991, 279),
                   (1018, 262), (1045, 245), (1073, 228), (1100, 210), (1128, 193),
                   (1157, 177), (1186, 159), (1215, 141), (1244, 123), (1273, 104)]

# Plot points in image
image = cv2.imread("undistExternalImg.png")
circleColor = (0, 0, 255)
# Add circles at each coordinate
for (x, y) in coordinatesList:
    cv2.circle(image, (x, y), 3, circleColor, -1)

cv2_imshow(image)

# Plot coords
xCoords, yCoords = zip(*coordinatesList)
plt.scatter(xCoords, yCoords, c='red', marker='o', s=50)
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.title('Pixel coordinates')

# Show the scatter plot
plt.show()

distancesX = [abs((x2 - x1)) for (x1, _), (x2, _) in zip(coordinatesList, coordinatesList[1:])]
distancesY = [abs((y2 - y1)) for (_, y1), (_, y2) in zip(coordinatesList, coordinatesList[1:])]

# Print the distances
#print(distancesX)
#print(distancesY)

# Plot for X
xCoords = [x for x, _ in coordinatesList]

# Perform linear regression on x-coordinates and distances
SLOPEX, INTERCEPTX, rValueX, pValueX, stdErrX = linregress(xCoords[1:], distancesX)
# Create the regression line
regressionLineX = [SLOPEX * x + INTERCEPTX for x in xCoords[1:]]
# Create a graph relating x-coordinates to distances with the regression line
plt.plot(xCoords[1:], distancesX, marker='o', linestyle='-', color='b', label='Data')
plt.plot(xCoords[1:], regressionLineX, color='r', linestyle='--', label=f'Regression Line (R-squared={rValueX**2:.2f})')
plt.xlabel('X Coordinate')
plt.ylabel('Distance X in pixels (x2-x1)')
plt.title('Relation Between X Coordinate and Distance')
plt.legend()
plt.grid(True)
# Show the plot with the regression line
plt.show()

#Plot for Y
# Extract x-coordinates from the coordinates list
yCoords = [y for _, y in coordinatesList]

# Perform linear regression on x-coordinates and distances
SLOPEY, INTERCEPTY, rValueY, pValueY, stdErrY = linregress(yCoords[1:], distancesY)
# Create the regression line
regressionLineY = [SLOPEY * y + INTERCEPTY for y in yCoords[1:]]
# Create a graph relating x-coordinates to distances with the regression line
plt.plot(yCoords[1:], distancesY, marker='o', linestyle='-', color='b', label='Data')
plt.plot(yCoords[1:], regressionLineY, color='r', linestyle='--', label=f'Regression Line (R-squared={rValueY**2:.2f})')
plt.xlabel('Y Coordinate')
plt.ylabel('Distance Y in pixels (y2-y1)')
plt.title('Relation Between Y Coordinate and Distance')
plt.legend()
plt.grid(True)
# Show the plot with the regression line
plt.show()

In [ ]:
def getPixelToMillimeterRatio(x_coordinate, y_coordinate):
  # Load the linear regression results from the JSON file
  xDistance = SLOPEX * x_coordinate + INTERCEPTX
  yDistance = SLOPEY * y_coordinate + INTERCEPTY

  oneCmInPixels = math.sqrt(xDistance**2 + yDistance**2)

  PixelToMillimeterRatio = oneCmInPixels / 10

  return PixelToMillimeterRatio

# **Exercise c)**
Consider only the images retrieved from single_coins. Implement the functions described above.


In [ ]:
class ROI:
  def __init__(self, imageROI, offsetX, offsetY, name=None):
    self.imageROI = imageROI
    self.offsetX = offsetX
    self.offsetY = offsetY
    self.name = name

  def getName(self):
    return self.name

In [ ]:
def detectROI(image, imageName):
  grayImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  # Detect background (cv2.THRESH_BINARY) using cv2.THRESH_OTSU that determines the optimal threshold value using the Otsu's or Triangle algorithm

  retval, backgroundImage = cv2.threshold(grayImage, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
  contours, hierarchy = cv2.findContours(backgroundImage, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  plotTwoImages(image, backgroundImage, "Image", "Background Image")

  # Detect biggest contour = paper
  maxContour = max(contours, key=cv2.contourArea)

  # Crop image knowing roi
  offSetX, offSetY, w, h = cv2.boundingRect(maxContour)
  roiImage = image.copy()
  roiImage = roiImage[offSetY:offSetY+h, offSetX:offSetX+w]

  roi = ROI(roiImage, offSetX, offSetY, imageName)

  return roi

In [ ]:
singleCoinsPaths = getImagesPathsList('/content/Assignment1/Single_coins')
allSingleCoinsRoi = []

for singleCoinsPath in singleCoinsPaths:
    singleCoinsImg = cv2.imread(singleCoinsPath)

    # Correct camera distortion
    undistSingleCoinImg = correctCameraDistortion(singleCoinsImg)

    # Print image name (if available)
    imageName = singleCoinsPath.split('/')[-1]
    print('Image:', imageName)

    # Display original and undistorted images
    plotTwoImages(singleCoinsImg, undistSingleCoinImg, "Distorted", "Undistorted")

    # Detect Region of Interest (ROI)
    singleCoinsRoi = detectROI(undistSingleCoinImg, imageName)

    # Display undistorted and ROI images
    plotTwoImages(undistSingleCoinImg, singleCoinsRoi.imageROI, "Undistorted", "ROI")
    print('\n\n---------------------------------------END-OF-IMAGE------------------------------------------\n\n')


    allSingleCoinsRoi.append(singleCoinsRoi)


**Coin Detection Functions.**
\
**Watershed trasnform was used.**

In [ ]:
def watershed(image, sureForgroundTresh, kernel, closeIterations = 1, dilateIterations = 1, openIterations = 0, threshold = None):

  grayImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  plotTwoImages(image, grayImage, "Image", "Gray Image")

  # Detect foreground (cv2.THRESH_BINARY_INV) using cv2.THRESH_OTSU that determines the optimal threshold value using the Otsu's or Triangle algorithm
  if  threshold == None:
    retval, foregroundImage = cv2.threshold(grayImage, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
  else:
    retval, foregroundImage = cv2.threshold(grayImage, treshold[0], treshold[1], cv2.THRESH_BINARY_INV)
  plotTwoImages(image, foregroundImage, "Image", "Foreground Image")

  # Apply close morph to fill gaps in coins

  morphImage = cv2.morphologyEx(foregroundImage, cv2.MORPH_CLOSE, kernel, iterations=closeIterations) # i = 2
  plotTwoImages(image, morphImage, "Image", "Morph Image (Close)")

  morphImage = cv2.morphologyEx(morphImage, cv2.MORPH_OPEN, kernel, iterations = openIterations)
  plotTwoImages(image, morphImage, "Image", "Morph Image (Open)")

  # Find sure background
  sureBackground = cv2.dilate(morphImage, kernel, iterations=dilateIterations) # i = 3
  plotTwoImages(image, sureBackground, "Image", "Sure Background (Dilate)")


  # Find sure foreground
  # Distance transform gives a gray iamge thats where value increses with less distance to nearest edge
  # So 255 is where the edge is
  distTransform = cv2.distanceTransform(morphImage, cv2.DIST_L2, 5) # DIST_L2 simple euclidean distance

  # Sureforeground represents the confidence of the foreground with respect to the edge via sureForgroundTresh.
  # If sureForgroundTresh is 0.05 it means that my sure foreground is everything after 5% distance from the edge
  # In this threshold I compare how certain I am that Im in the sure foreground the closest I get to the edge
  retval, sureForeground = cv2.threshold(distTransform, sureForgroundTresh*distTransform.max(), 255,0)
  plotTwoImages(image, sureForeground, "Image", "Sure Foreground (distTransfrom)")

  # Find unknwon region
  sureForeground = np.uint8(sureForeground)
  unkownRegion = cv2.subtract(sureBackground, sureForeground)
  plotTwoImages(image, unkownRegion, "Image", "Unkown Region (surebg - surefg)")

  detectedCoinsImage = image

  # Marker labelling
  retval, markers = cv2.connectedComponents(sureForeground)
  # Add one to all labels so that sure background is not 0, but 1
  markers = markers + 1

  # Set unkown region marker to zero
  markers[unkownRegion==255] = 0

  markers = cv2.watershed(detectedCoinsImage, markers)

  # Plot the labeled image using the custom colormap
  cmap = ListedColormap([plt.cm.tab20(i) for i in range(1, markers.max() + 1)])
  plt.close('all')
  plt.figure(figsize=(12, 6))
  plt.imshow(markers, cmap=cmap)
  plt.title("Markers")
  plt.axis('off')
  plt.show()

  return markers

In [ ]:
def checkIfCircle(contour, min_circularity=0.35, max_circularity=1.0):
    # Calculate the circularity of the contour
    perimeter = cv2.arcLength(contour, True)
    area = cv2.contourArea(contour)
    if perimeter == 0:
        return False
    circularity = 4 * np.pi * (area / (perimeter * perimeter))

    # Adjust the circularity threshold
    if min_circularity <= circularity <= max_circularity:
      return True
    #print("rejected circularity: " + str(circularity))
    return False

# Area in pixels
def detectCoinContours(image, whatershedMarkers, areaMargin, printID = False):

  # Define aspect ratio margin to later check if contour is circle
  # aspectRatioMargin = 0.3

  # Define margin percentage for contours area, having into account smallest and biggest coin area, also the pixel to mm ratio with coordinates
  minRadius = getPixelToMillimeterRatio(0, 600) * min(allPerfectDiameters)/2
  maxRadius = getPixelToMillimeterRatio(1600, 0) * max(allPerfectDiameters)/2
  minContourArea = (1 - areaMargin) * circleArea(minRadius)
  maxContourArea = (1 + areaMargin) * circleArea(maxRadius)

  contourImage = image.copy()
  contourImageToPlot = image.copy()
  allContoursImage = image.copy()


  coinsContour = []
  grayImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  coinID = 1

  for marker in np.unique(watershedMarkers):
    # Skip background
    if marker == 0:
      continue

    markerMask = np.zeros_like(grayImage, dtype=np.uint8)
    markerMask[whatershedMarkers == marker] = 255

    # Detect countour of current marker only
    contours, hierarchy = cv2.findContours(markerMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


    for contour in contours:

      contourArea = cv2.contourArea(contour)
      cv2.drawContours(allContoursImage, [contour], -1, (0, 255, 0), thickness=5)

      if checkIfCircle(contour) == True and contourArea > minContourArea and contourArea < maxContourArea:

        # Coin contour image
        cv2.drawContours(contourImage, [contour], -1, (0, 0, 0), thickness=1)
        coinsContour.append(contour)

        cv2.drawContours(contourImageToPlot, [contour], -1, (0, 255, 0), thickness=5)

        # Print coin ID
        if printID == True:
          x, y, w, h = cv2.boundingRect(contour)
          cv2.putText(contourImageToPlot, "ID: " + str(coinID), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
          coinID += 1

  plotTwoImages(allContoursImage, contourImageToPlot, "All contours", "Contour Image to plot")
  #plotTwoImages(image, contourImageToPlot, "Image", "Contour Image to plot")


  return coinsContour

In [ ]:
notChecked = 'Not Checked'
allPerfectDiameters = [16.25, 18.75, 21.25, 19.75, 22.25, 24.25, 23.25, 25.75]
coinTypeList = ["1c", "2c", "5c", "10c", "20c", "50c", "1Eur", "2Eur"]

# Area must be in mm^2
class Coin:
  def __init__(self, area, checkCoinType, pixelTommRatio, extractedCoinImg, checkWithHist):
    self.area = area
    self.diameter = self.calculateDiameter(area)
    self.pixelTommRatio = pixelTommRatio
    self.extractedCoinImg = extractedCoinImg
    self.checkWithHist = checkWithHist

    self.coinType = notChecked
    if checkCoinType == True:
      self.coinType = self.matchCoinType(self.diameter, self.extractedCoinImg, self.checkWithHist)

  def calculateDiameter(self, area):

    radius = (area / math.pi) ** 0.5
    diameter = radius * 2
    return diameter

  def matchCoinType(self, diameter, extractedCoinImg, checkWithHist):

    hist = calcHistogramHue(extractedCoinImg, plot=False)
    #plt.figure(figsize=(4,4))
    #plt.imshow(cv2.cvtColor(extractedCoinImg, cv2.COLOR_BGR2RGB))
    #plt.show()
    indexs = [0, 7]
    if checkWithHist == True:
      indexs = calcCheckCoinIndexs(hist, bronzeHist, goldHist, silverGoldHist)

    closestDiameterIndex = indexs[0]

    # Find closest match between detected diameter and real diameter
    # Check limited index range, limited using histogram
    for i in range(indexs[0], indexs[1]+1): #indexs[1]+1, +1 cuz range would stop at one before
      perfectDiameter = allPerfectDiameters[i]

      if(abs(perfectDiameter - diameter) < abs(allPerfectDiameters[closestDiameterIndex] - diameter)):
        closestDiameterIndex = i

    # Check 2 euro coins if its not bronze or gold, since i dont have its histogram
    if(indexs != [0, 2] and indexs != [3, 5] and abs(allPerfectDiameters[7] - diameter) <  abs(allPerfectDiameters[closestDiameterIndex] - diameter)):
      closestDiameterIndex = 7


    # If using the histogram there is a big error, repeat the checking without histogram index limitation
    if(abs(allPerfectDiameters[closestDiameterIndex] - diameter) >= 1): #1 #1.2
      for i in range(0, 7+1): #7+1, +1 cuz range would stop at one before
        perfectDiameter = allPerfectDiameters[i]

        if(abs(perfectDiameter - diameter) < abs(allPerfectDiameters[closestDiameterIndex] - diameter)):
          closestDiameterIndex = i


    return coinTypeList[closestDiameterIndex]

  def print(self):

    areaInPixels = circleArea(self.diameter/2 * self.pixelTommRatio)

    print("Area: ", areaInPixels , "pixels")
    print("Area: ", self.area, " mm^2")
    print("Diameter: ", self.diameter, " mm")

    if self.coinType != notChecked:
      print("Type: ", self.coinType)

      # Plot extracted coin img
      #plt.figure(figsize=(4,4))
      #plt.imshow(cv2.cvtColor(self.extractedCoinImg, cv2.COLOR_BGR2RGB))
      #plt.show()

  def getValue(self):
    coinValue = 0

    if self.coinType == coinTypeList[0]:
      coinValue = 0.01
    elif self.coinType == coinTypeList[1]:
      coinValue = 0.02
    elif self.coinType == coinTypeList[2]:
      coinValue = 0.05
    elif self.coinType == coinTypeList[3]:
      coinValue = 0.10
    elif self.coinType == coinTypeList[4]:
      coinValue = 0.20
    elif self.coinType == coinTypeList[5]:
      coinValue = 0.50
    elif self.coinType == coinTypeList[6]:
      coinValue = 1.00
    elif self.coinType == coinTypeList[7]:
      coinValue = 2.00

    return coinValue



def detectCoins(coinContours, checkCoinType, roi, checkWithHist):

  detectedCoins = []
  imageToDraw = roi.imageROI.copy()

  for contour in coinContours:

    x, y, w, h = cv2.boundingRect(contour)
    M = cv2.moments(contour)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])

    pixelTommRatio = getPixelToMillimeterRatio(cX + roi.offsetX, cY + roi.offsetY)
    #print("cx:", cX, "offsetX: ", roi.offsetX, "sum: ", cX + roi.offsetX, "PixelTommRatio: ", pixelTommRatio)

    contourAreaPixels = cv2.contourArea(contour)
    contourAreaMili = contourAreaPixels / (pixelTommRatio**2)


    extractedCoinOnlyImg = extractCoinsOnlyImage(roi.imageROI, [contour])

    coin = Coin(contourAreaMili, checkCoinType, pixelTommRatio, extractedCoinOnlyImg, checkWithHist)
    detectedCoins.append(coin)

    if checkCoinType == True:
      cv2.rectangle(imageToDraw, (x, y), (x + w, y + h), (0, 255, 0), 3)
      cv2.putText(imageToDraw, coin.coinType, (x + 2, y + int(h/2)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3, cv2.LINE_AA)

      #cv2.putText(imageToDraw, str(round(calcDiameter(contourAreaMili),2)), (x + 2, y + int(h)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (0, 255, 0), 1, cv2.LINE_AA)



  if checkCoinType == True:
    plt.close('all')
    plt.figure(figsize=(12, 5))
    plt.imshow(cv2.cvtColor(imageToDraw, cv2.COLOR_BGR2RGB))
    plt.title("Detected Coins")
    plt.axis('off')
    plt.show()

  return detectedCoins

def extractCoinsOnlyImage(image, coinContours):

  mask = np.zeros_like(image)

  # Draw the contours on the mask and fill them in (cv2.FILLED)
  cv2.drawContours(mask, coinContours, -1, (255, 255, 255), thickness=cv2.FILLED)

  # Apply mask to image
  coinsOnlyImage = cv2.bitwise_and(image, mask)

  return coinsOnlyImage

def calcHistogramHue(image, coinNum = 1, plot = False):
  coinsOnlyImageHSV = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
  hueChannel = coinsOnlyImageHSV[:, :, 0]
  hueChannelHist = cv2.calcHist([hueChannel], [0], None, [256], [1, 256]) / coinNum # [1, 256] 1 to remove pure black, devide by coinNum to normalize

  # Plot the histogram of hue
  if plot == True:
    plt.plot(hueChannelHist, color='b')
    plt.title('Hue Histogram normalized by number of coins')
    plt.xlabel('Hue Value')
    plt.ylabel('Frequency')
    plt.show()

  return hueChannelHist

def combineHists(histList):

  sumHist = np.array(histList[0], dtype=np.float64)

  if len(histList) > 1:
    for hist in histList[1:]:
      sumHist += np.array(hist, dtype=np.float64)

  numHist = len(histList)
  averageHist = sumHist / numHist

  return averageHist

# Difference of the sum of the values in the histograms
def histDifference(hist1, hist2):
  return np.sum(np.abs(hist1 - hist2))

# Check which histogram is closest and limit the search to the corresponding coin indexs
def calcCheckCoinIndexs(hist, bronzeHist, goldHist, silverGoldHist):
  bronzeHistDiff = histDifference(hist, bronzeHist)
  goldHistDiff = histDifference(hist, goldHist)
  silverGoldHistDiff = histDifference(hist, silverGoldHist)


  indexs = [0, 7]

  if bronzeHistDiff < goldHistDiff and bronzeHistDiff < silverGoldHistDiff:
    indexs = [0, 2]
    #print("LIMIT TO BRONZE")
  elif goldHistDiff < bronzeHistDiff and goldHistDiff < silverGoldHistDiff:
    indexs = [3, 5]
    #print("LIMIT TO GOLD")
  else:
    indexs = [6, 6] # Only limit to 1 euro
    #print("LIMIT TO 1EUR")

  # Plot the histograms for comparison
  #plt.plot(hist, color='b', label="Detected Histogram")
  #plt.plot(bronzeHist, color='g', label="Bronze Histogram")
  #plt.plot(goldHist, color='r', label="Gold Histogram")
  #plt.plot(silverGoldHist, color='y', label="Silver Gold Histogram")
  #plt.title('Hue Histogram Comparison')
  #plt.xlabel('Hue Value')
  #plt.ylabel('Frequency')
  #plt.legend()
  #plt.show()

  return indexs


In [ ]:
sureForegroundTreshold = 0.03
areaMargin = 0.5
checkCoinType = False  # In this exercise, all the coins in the image are the same
checkWithHist = False # DONT SET TO TRUE, HISTOGRAM OF BRONZE GOLD AND SILVER DOESNT YET EXIST


kernel = np.ones((5, 5), dtype=int)  # cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernelDiameter + 1, kernelDiameter + 1))
print(kernel)

# Color type hist list
bronzeHists = []
goldHists = []
silverGoldHists = []


i = 1

for singleCoinsRoi in allSingleCoinsRoi:
  print('Image:', singleCoinsRoi.getName())

  watershedMarkers = watershed(singleCoinsRoi.imageROI, sureForegroundTreshold, kernel)
  coinContours = detectCoinContours(singleCoinsRoi.imageROI, watershedMarkers, areaMargin, True)

  coins = detectCoins(coinContours, checkCoinType, singleCoinsRoi, checkWithHist)
  coinsDiameters = []

  coinsOnlyImage = extractCoinsOnlyImage(singleCoinsRoi.imageROI, coinContours)
  plotTwoImages(singleCoinsRoi.imageROI, coinsOnlyImage, "Image", "Coins Only Image")

  j = 1

  for coin in coins:
      coinsDiameters.append(coin.diameter)
      print("Coin ID:", j)
      coin.print()
      j += 1

  # Print exercise stats
  print("\n\nDiameter mean:", np.mean(coinsDiameters), "mm")
  print("Diameter std:", np.std(coinsDiameters), "mm")
  hueHist = calcHistogramHue(coinsOnlyImage, len(coinContours), True)

  # Insert hist into dictionary
  if singleCoinsRoi.getName() == 'img_1cents.png' or singleCoinsRoi.getName() == 'img_2cents.png':
    bronzeHists.append(hueHist)
  elif singleCoinsRoi.getName() == 'img_20cents.png':
    goldHists.append(hueHist)
  elif singleCoinsRoi.getName() == 'img_1euro.png':
    silverGoldHists.append(hueHist)

  i += 1
  print('\n\n---------------------------------------END-OF-IMAGE------------------------------------------\n\n')

# Do the average of the hists for checking when diciding the coin type
bronzeHist = combineHists(bronzeHists)
goldHist = combineHists(goldHists)
silverGoldHist = combineHists(silverGoldHists)

# Plot hist of color type
plt.figure(figsize=(24, 6))
plt.subplot(131)
plt.plot(bronzeHist, color='b')
plt.title('Bronze Hist (Average histogram of 1 and 2 cents)')
plt.xlabel('Hue Value')
plt.ylabel('Frequency')
plt.subplot(132)
plt.plot(goldHist, color='b')
plt.title('Gold Hist (Same has 20cent histogram)')
plt.xlabel('Hue Value')
plt.ylabel('Frequency')
plt.subplot(133)
plt.plot(silverGoldHist, color='b')
plt.title('Silver-Gold Hist (Same has 1 euro histogram)')
plt.xlabel('Hue Value')
plt.ylabel('Frequency')
plt.show()

As printed by the code:
\
Image: img_20cents.png:
\
Diameter mean: 22.4076721181734 mm
\
Diameter std: 0.4222284237321345 mm
\
\
Image: img_1cents.png:
\
Diameter mean: 16.356879393799346 mm
\
Diameter std: 0.25085579524598484 mm
\
\
Image: img_1euro.png:
\
Diameter mean: 23.835212489584872 mm
\
Diameter std: 0.2868338695845823 mm
\
\
Image: img_2cents.png:
\
Diameter mean: 18.58571774581288 mm
\
Diameter std: 0.19039705054062922 mm

# **Exercise d)**
Consider only the images retrieved from multiple_coins. Implement the
functions described above.

In [ ]:
filenames = ["img_5_2_1cents", "img_50_20_10cents", "img_50_20cents"]

basePath = '/content/Assignment1/Multiple_coins/'

# Use glob to find the matching image files
multipleCoinsPaths = []

for filename in filenames:
    filePath = basePath + filename + '.png'
    matches = glob.glob(filePath)
    multipleCoinsPaths.extend(matches)

allMultipleCoinsRoi = []

for multipleCoinsPath in multipleCoinsPaths:
  imageName = getNameFromPath(multipleCoinsPath)
  print(imageName)

  multipleCoinsImg = cv2.imread(multipleCoinsPath)

  undistMultipleCointImg = correctCameraDistortion(multipleCoinsImg)

  plotTwoImages(multipleCoinsImg, undistMultipleCointImg, "Distorted", "Undistorted")
  multipleCoinsRoi = detectROI(undistMultipleCointImg, imageName)
  plotTwoImages(undistMultipleCointImg, multipleCoinsRoi.imageROI, "Undistorted", "ROI")
  print('\n\n---------------------------------------END-OF-IMAGE------------------------------------------\n\n')


  allMultipleCoinsRoi.append(multipleCoinsRoi)

Note that the code to detect the coin type is inside the coin class. The code to plot the image is inside the detectCoins() function, when checkCoinType is True.

In [ ]:
def countMoney(coins):
  money = 0

  for coin in coins:
    money += coin.getValue()

  return money

In [ ]:
sureForgroundTresh = 0.05
areaMargin = 0.5
checkCoinType = True
checkWithHist = True

kernel = np.ones((5, 5), dtype=int)
print(kernel)

i = 1

for multipleCoinsRoi in allMultipleCoinsRoi:
  print(multipleCoinsRoi.getName())

  coinTypeCounts = defaultdict(int)

  watershedMarkers = watershed(multipleCoinsRoi.imageROI, sureForgroundTresh, kernel)
  coinContours = detectCoinContours(multipleCoinsRoi.imageROI, watershedMarkers, areaMargin, True)

  coins = detectCoins(coinContours, checkCoinType, multipleCoinsRoi, checkWithHist)

  money = countMoney(coins)

  j = 1
  for coin in coins:
    coinTypeCounts[coin.coinType] += 1
    #print("Coin ID:", j)
    #coin.print()
    j+=1
  print("Total money: ", money, "€")

  i+=1

  # Plot histogram of coins
  coinTypes = list(coinTypeCounts.keys())
  counts = [coinTypeCounts[coinType] for coinType in coinTypes]

  plt.bar(coinTypes, counts)
  plt.xlabel('Coin Type')
  plt.ylabel('Count')
  plt.title('Number of Coins per Coin Type')
  plt.show()
  print('\n\n---------------------------------------END-OF-IMAGE------------------------------------------\n\n')


# **Exercise e)**
Repeate the previous point to images “img_coins_small” and “img_coins2_large”.

In [ ]:
filenames2 = ["img_coins_small", "img_coins2_large"]

basePath = '/content/Assignment1/Multiple_coins/'

# Use glob to find the matching image files
multipleCoinsPaths2 = []

for filename in filenames2:
    filePath2 = basePath + filename + '.png'
    matches = glob.glob(filePath2)
    multipleCoinsPaths2.extend(matches)

allMultipleCoinsRoi2 = []

for multipleCoinsPath in multipleCoinsPaths2:
  imageName = getNameFromPath(multipleCoinsPath)
  print(imageName)

  multipleCoinsImg = cv2.imread(multipleCoinsPath)

  undistMultipleCointImg = correctCameraDistortion(multipleCoinsImg)

  plotTwoImages(multipleCoinsImg, undistMultipleCointImg, "Distorted", "Undistorted")

  multipleCoinsRoi = detectROI(undistMultipleCointImg, imageName)
  plotTwoImages(undistMultipleCointImg, multipleCoinsRoi.imageROI, "Undistorted", "ROI")
  print('\n\n---------------------------------------END-OF-IMAGE------------------------------------------\n\n')

  allMultipleCoinsRoi2.append(multipleCoinsRoi)

In [ ]:
sureForgroundTresh = 0.25#0.25#0.25#0.26#0.26#0.27
areaMargin = 0.5
checkCoinType = True # In this exercise all the coins in the image are the same
checkWithHist = True

kernel = np.ones((5, 5), dtype=int)
print(kernel)
closeInterations = 1#2
openIterations = 0#4 # 3
dilateIterations = 1#3

treshold = [169, 255]

i = 1

for multipleCoinsRoi in allMultipleCoinsRoi2:
  print(multipleCoinsRoi.getName())

  coinTypeCounts = defaultdict(int)

  watershedMarkers = watershed(multipleCoinsRoi.imageROI, sureForgroundTresh, kernel, closeInterations, dilateIterations, openIterations, treshold)
  coinContours = detectCoinContours(multipleCoinsRoi.imageROI, watershedMarkers, areaMargin)

  coins = detectCoins(coinContours, checkCoinType, multipleCoinsRoi, checkWithHist)

  money = countMoney(coins)

  j = 1
  for coin in coins:

    coinTypeCounts[coin.coinType] += 1
    j+=1

  print("Total money: ", money, "€")

  i+=1

  # Plot histogram of coins
  coinTypes = list(coinTypeCounts.keys())
  counts = [coinTypeCounts[coinType] for coinType in coinTypes]

  plt.bar(coinTypes, counts)
  plt.xlabel('Coin Type')
  plt.ylabel('Count')
  plt.title('Number of Coins per Coin Type')
  plt.show()
  print('\n\n---------------------------------------END-OF-IMAGE------------------------------------------\n\n')


# **Exercise f)**
Consider the image “img_coins_nbackground” that depicts many coins captured without a white background. Repeat all the point d) without defining a region of interest.

Note that ROI class is used but only beacause the rest of the code requires it. The offsets of x and y are zero since no cropping is done.

In [ ]:
filenames3 = ["img_coins_nbackground"]

basePath = '/content/Assignment1/NoBackground_coins/'


# Use glob to find the matching image files
multipleCoinsPaths3 = []

tableHsvLower = (0, 0, 0)
tableHsvUpper = (255, 45, 255)#(255, 30, 255)

darkCoinsHsvLower = (113, 34, 0)
darkCoinsHsvUpper = (255, 255, 255)

gaussianKernel = (9, 9)
gaussianSigma = 1

for filename in filenames3:
    filePath = basePath + filename + '.png'
    matches = glob.glob(filePath)
    multipleCoinsPaths3.extend(matches)

allMultipleCoins3 = []

for multipleCoinsPath in multipleCoinsPaths3:

  imageName = getNameFromPath(multipleCoinsPath)
  print(imageName)

  multipleCoinsImg = cv2.imread(multipleCoinsPath)

  undistMultipleCointImg = correctCameraDistortion(multipleCoinsImg)
  plotTwoImages(multipleCoinsImg, undistMultipleCointImg, "Distorted", "Undistorted")

  # Apply gaussian blurr to remove high frequency noise
  undistMultipleCointImgBlured = cv2.GaussianBlur(src=undistMultipleCointImg, ksize=gaussianKernel , sigmaX=gaussianSigma)
  undistMultipleCointImg = undistMultipleCointImgBlured

  undistMultipleCointImgHSV = cv2.cvtColor(undistMultipleCointImg, cv2.COLOR_BGR2HSV)

  # Table mask
  maskTable = cv2.inRange(undistMultipleCointImgHSV, tableHsvLower, tableHsvUpper)
  tableImg = cv2.bitwise_and(undistMultipleCointImg, undistMultipleCointImg, mask=maskTable)
  plotTwoImages(undistMultipleCointImg, tableImg, "Undistorted", "Table")

  # Dark coins mask
  maskDarkCoins = cv2.inRange(undistMultipleCointImgHSV, darkCoinsHsvLower, darkCoinsHsvUpper)
  darkCoinsImg = cv2.bitwise_and(undistMultipleCointImg, undistMultipleCointImg, mask=maskDarkCoins)
  plotTwoImages(undistMultipleCointImg, darkCoinsImg, "Undistorted", "Dark Coins")

  # Only coins Image = !Table mask OR Dark coins mask
  noTableImg = undistMultipleCointImg - tableImg
  plotTwoImages(undistMultipleCointImg, noTableImg, "Undistorted", "No table")

  onlyCoinsImg = cv2.bitwise_or(noTableImg, darkCoinsImg)
  plotTwoImages(undistMultipleCointImg, onlyCoinsImg, "Undistorted", "Only Coins (No table + Dark coins)")

  # Set background from black to white, since Whatershed framework assumes a white background
  blackMask = np.all(onlyCoinsImg == (0, 0, 0), axis=2)
  onlyCoinsImg[blackMask] = (255, 255, 255)
  plotTwoImages(undistMultipleCointImg, onlyCoinsImg, "Undistorted", "Only Coins White Background")

  noTableImgRoi = ROI(onlyCoinsImg, 0, 0, imageName)

  allMultipleCoins3.append(noTableImgRoi)


In [ ]:
sureForgroundTresh = 0.22#0.22#0.25#0.2
areaMargin = 0.5
checkCoinType = True # In this exercise all the coins in the image are the same
checkWithHist = True


kernel = np.ones((5, 5), dtype=int)
print(kernel)
closeInterations = 2
openIterations = 0#4 # 3
dilateIterations = 1

treshold = [240, 255]


for multipleCoins in allMultipleCoins3:

  print(multipleCoinsRoi.getName())

  coinTypeCounts = defaultdict(int)

  watershedMarkers = watershed(multipleCoins.imageROI, sureForgroundTresh, kernel, closeInterations, dilateIterations, openIterations, treshold)
  coinContours = detectCoinContours(multipleCoins.imageROI, watershedMarkers, areaMargin)

  coins = detectCoins(coinContours, checkCoinType, multipleCoins, checkWithHist)


  money = countMoney(coins)

  j = 1
  for coin in coins:

    coinTypeCounts[coin.coinType] += 1
    j+=1



  print("Total money: ", money, "€")

  i+=1

  # Plot histogram of coins
  coinTypes = list(coinTypeCounts.keys())
  counts = [coinTypeCounts[coinType] for coinType in coinTypes]

  plt.bar(coinTypes, counts)
  plt.xlabel('Coin Type')
  plt.ylabel('Count')
  plt.title('Number of Coins per Coin Type')
  plt.show()
  print('\n\n---------------------------------------END-OF-IMAGE------------------------------------------\n\n')

#**Exercise g)**
Discussion: address the challenge of noise and complex backgrounds, including the properties of the hardware that was used for this Coinoscope and the presence of distracting elements that need to be filtered out to ensure accurate coin detection. Provide some suggestions to improve the performance of the Coinoscope (e.g., imaging setup, calibration process and photometric effects).

**Answer:**

The main chalange was the very small difference between most coins, arround 1 mm.
\
\
This problem was made worse by bad lighting conditions and the camera point of view.
\
\
The bad lighting made the thresholding more difficult, resulting in thresholded coins with gaps in them. To address this issue morphological operations were used, however sometimes these could not entierly be filled without losing the original image features. The coins shadows could also enlarge the detected coin area, resulting in missclassifications in coins with similar diameters. Mainly in the last exercise, bad lighting resulted in the table a silver reflection, thus having similar colors to the 2 euro coins. Fortunatilly, the coin contour filtering eliminated these contours.
\
\
The camera point of view resulted in a non constant pixel to millimiter ratio, which induced error when the coins were far from the liniariztion arround the ruller.
\
\
To improve the setup thse probelms should be addressed at their root. The camera point of view should be parallel to the table. The image used to obtain the pixel to millimiter ratio should be an image with a chessboard with a known square size and their corners detected automaticlly, to eliminate the error when selecting the pixels on the ruller. These would fix the error in the diameter calculations.
\
\
To remove bad lighthing a strong backlight should be used. This way no shadows would exist and a constant white backgroud would highlight the coins silhouettes. This way it would be easier to find the contours, even when coins were very close togheter.  